# SageMaker script mode
> How to load SageMaker builtin Linear Learner model with Apache MXNet in Python.

- toc: true 
- badges: true
- comments: true
- categories: [aws, ml, sagemaker]
- keyword: [aws, ml, sagemaker, apache, mxnet, gluon]
- image: images/copied_from_nb/images/2022-07-07-sagemaker-script-mode.jpeg

![](images/2022-07-07-sagemaker-script-mode.jpeg)

In [2]:
import boto3
import pandas as pd
import numpy as np

s3 = boto3.client("s3")
s3.download_file(f"sagemaker-sample-files", "datasets/tabular/iris/iris.data", "iris.data")

df = pd.read_csv(
    "iris.data", header=None, names=["sepal_len", "sepal_wid", "petal_len", "petal_wid", "class"]
)
df.head()

,sepal_len,sepal_wid,petal_len,petal_wid,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
# Convert the three classes from strings to integers in {0,1,2}
df["class_cat"] = df["class"].astype("category").cat.codes
categories_map = dict(enumerate(df["class"].astype("category").cat.categories))
print(categories_map)
df.head()

{0: 'Iris-setosa', 1: 'Iris-versicolor', 2: 'Iris-virginica'}


,sepal_len,sepal_wid,petal_len,petal_wid,class,class_cat
0,5.1,3.5,1.4,0.2,Iris-setosa,0
1,4.9,3.0,1.4,0.2,Iris-setosa,0
2,4.7,3.2,1.3,0.2,Iris-setosa,0
3,4.6,3.1,1.5,0.2,Iris-setosa,0
4,5.0,3.6,1.4,0.2,Iris-setosa,0


In [4]:
# Split the data into 80-20 train-test split
num_samples = df.shape[0]
split = round(num_samples * 0.8)
train = df.iloc[:split, :]
test = df.iloc[split:, :]
print("{} train, {} test".format(split, num_samples - split))

120 train, 30 test


In [5]:
local_data_loc = "./datasets/2022-07-07-sagemaker-script-mode/"

In [6]:
# Write train and test CSV files
train.to_csv(f"{local_data_loc}train.csv", index=False)
test.to_csv(f"{local_data_loc}test.csv", index=False)

In [7]:
import sagemaker

session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = session.default_bucket()
region = session.boto_region_name

print(f"sagemaker.__version__: {sagemaker.__version__}")
print(f"Session: {session}")
print(f"Role: {role}")
print(f"Bucket: {bucket}")
print(f"Region: {region}")

sagemaker.__version__: 2.73.0
Session: <sagemaker.session.Session object at 0x7fa487e97760>
Role: arn:aws:iam::801598032724:role/service-role/AmazonSageMaker-ExecutionRole-20220516T161743
Bucket: sagemaker-us-east-1-801598032724
Region: us-east-1


In [9]:
from sagemaker.local import LocalSession
session_local = LocalSession()
session_local

In [15]:
session_local.config = {'local': {'local_code': True}}

In [ ]:
# estimator = sagemaker.estimator.Estimator(
# container,
# role,
# instance_count=1,
# instance_type='local',
# output_path=training_s3_output_location,
# sagemaker_session=session)

In [13]:
%%writefile $local_data_loc/train.py

if __name__ == "__main__":
    print("sagemaker train")


Writing ./datasets/2022-07-07-sagemaker-script-mode//train.py


In [18]:
# !pip install 'sagemaker[local]' --upgrade
# this is required

In [19]:
from sagemaker.sklearn import SKLearn

sk_estimator = SKLearn(
    entry_point=f"{local_data_loc}train.py",
    role=role,
    instance_count=1,
    instance_type='local',
    py_version="py3",
    framework_version="1.0-1",
    script_mode=True,
    hyperparameters={"estimators": 20},
    sagemaker_session=session_local
)

In [22]:
sk_estimator.fit()

Using the short-lived AWS credentials found in session. They might expire while running.


FileNotFoundError: [Errno 2] No such file or directory: 'docker'